In [1]:
import pandas as pd
import numpy as np
import plot_params
import seaborn as sns
import matplotlib.pyplot as plt
import scipy.stats as stats

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import precision_recall_curve, auc
from sklearn.metrics import average_precision_score
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report
from sklearn.pipeline import Pipeline
from joblib import load

plot_params.apply_rcparams()

In [2]:
X_train = load('X_train.joblib')
y_train = load('y_train.joblib')
X_test = load('X_test.joblib')
y_test = load('y_test.joblib')

In [ ]:
logreg_pipeline = Pipeline([
   ('scaler', StandardScaler()),  # Scaling within the pipeline
    ('logreg', LogisticRegression(solver='saga', max_iter=500))
])

logreg_params = {
    'logreg__C': [0.1, 1, 10, 100],
    'logreg__penalty': ['l1', 'l2'],
}

In [4]:
svc_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('svc', LinearSVC())
])

svc_params = {
    'svc__C': [0.01, 0.1, 1, 10, 100],
}

In [11]:
rf_pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('rf', RandomForestClassifier(random_state=12))
])

rf_params = {
    'rf__n_estimators': [50, 200], 
    'rf__max_depth': [None, 10, 20],
}

In [6]:
grid_search_logreg = GridSearchCV(logreg_pipeline, logreg_params, cv=5, 
                                  scoring='f1_macro', n_jobs=-1, verbose=1)
grid_search_logreg.fit(X_train, y_train)
print("Best Logistic Regression Parameters:", grid_search_logreg.best_params_)
print("Best Logistic Regression Accuracy:", grid_search_logreg.best_score_)

Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Logistic Regression Parameters: {'logreg__C': 100, 'logreg__penalty': 'l2'}
Best Logistic Regression Accuracy: 0.9710417014063044


C:\Users\frang\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [7]:
random_search_logreg = RandomizedSearchCV(logreg_pipeline, logreg_params, 
                                          n_iter=20, cv=5, scoring='f1_macro', 
                                          n_jobs=-1, verbose=1, 
                                          random_state=42)
random_search_logreg.fit(X_train, y_train)
print("Best Randomized Logistic Regression Parameters:", 
      random_search_logreg.best_params_)
print("Best Randomized Logistic Regression Accuracy:", 
      random_search_logreg.best_score_)

C:\Users\frang\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 8 is smaller than n_iter=20. Running 8 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Fitting 5 folds for each of 8 candidates, totalling 40 fits
Best Randomized Logistic Regression Parameters: {'logreg__penalty': 'l2', 'logreg__C': 100}
Best Randomized Logistic Regression Accuracy: 0.9710417014063044


C:\Users\frang\AppData\Roaming\Python\Python39\site-packages\sklearn\linear_model\_sag.py:349: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(


In [8]:
grid_search_svc = GridSearchCV(svc_pipeline, svc_params, cv=5, 
                               scoring='f1_macro', n_jobs=-1, verbose=1)
grid_search_svc.fit(X_train, y_train)
print("Best SVC Parameters:", grid_search_svc.best_params_)
print("Best SVC Accuracy:", grid_search_svc.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits
Best SVC Parameters: {'svc__C': 10}
Best SVC Accuracy: 0.9701861747842166


In [9]:
random_search_svc = RandomizedSearchCV(svc_pipeline, svc_params, n_iter=50, 
                                       cv=5, scoring='f1_macro', n_jobs=-1, 
                                       verbose=1, random_state=42)
random_search_svc.fit(X_train, y_train)
print("Best Randomized SVC Parameters:", random_search_svc.best_params_)
print("Best Randomized SVC Accuracy:", random_search_svc.best_score_)

Fitting 5 folds for each of 5 candidates, totalling 25 fits


C:\Users\frang\AppData\Roaming\Python\Python39\site-packages\sklearn\model_selection\_search.py:320: UserWarning: The total space of parameters 5 is smaller than n_iter=50. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best Randomized SVC Parameters: {'svc__C': 10}
Best Randomized SVC Accuracy: 0.9701861747842166


In [ ]:
grid_search_rf = GridSearchCV(rf_pipeline, rf_params, cv=5, 
                              scoring='f1_macro', n_jobs=-1, verbose=1)
grid_search_rf.fit(X_train, y_train)
print("Best Random Forest Parameters:", grid_search_rf.best_params_)
print("Best Random Forest Accuracy:", grid_search_rf.best_score_)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


In [ ]:
random_search_rf = RandomizedSearchCV(rf_pipeline, rf_params, n_iter=20, 
                                      cv=5, scoring='f1_macro', n_jobs=-1, 
                                      verbose=1, random_state=12)
random_search_rf.fit(X_train, y_train)
print("Best Randomized Random Forest Parameters:", 
      random_search_rf.best_params_)
print("Best Randomized Random Forest Accuracy:", random_search_rf.best_score_)

In [ ]:
best_logreg = grid_search_logreg.best_estimator_ 
y_pred_logreg = best_logreg.predict(X_test)
print("\nLogistic Regression Test Performance:")
print(classification_report(y_test, y_pred_logreg))
print("Test Accuracy:", accuracy_score(y_test, y_pred_logreg))


Logistic Regression Test Performance:
              precision    recall  f1-score   support

           0       1.00      0.98      0.99     56864
           1       0.09      0.92      0.17        98

    accuracy                           0.98     56962
   macro avg       0.55      0.95      0.58     56962
weighted avg       1.00      0.98      0.99     56962

Test Accuracy: 0.9846739931884414


In [ ]:
best_svc = grid_search_svc.best_estimator_
y_pred_svc = best_svc.predict(X_test)
print("\nSVC Test Performance:")
print(classification_report(y_test, y_pred_svc))
print("Test Accuracy:", accuracy_score(y_test, y_pred_svc))

AttributeError: 'GridSearchCV' object has no attribute 'best_estimator_'

In [ ]:
best_rf = grid_search_rf.best_estimator_
y_pred_rf = best_rf.predict(X_test)
print("\nRandom Forest Test Performance:")
print(classification_report(y_test, y_pred_rf))
print("Test Accuracy:", accuracy_score(y_test, y_pred_rf))

NameError: name 'grid_search_rf' is not defined

model features which are interesting? precision recall curve just predicting postive values


In [ ]:
y_score_logreg = best_logreg.decision_function(X_test) 
precision_logreg, recall_logreg, _ = \
    precision_recall_curve(y_test, y_score_logreg)
auc_logreg = auc(recall_logreg, precision_logreg)
avg_prec_logreg = average_precision_score(y_test, y_score_logreg)

In [ ]:
y_score_svc = best_svc.decision_function(X_test)  
precision_svc, recall_svc, _ = \
    precision_recall_curve(y_test, y_score_svc)
auc_svc = auc(recall_svc, precision_svc)
avg_prec_svc = average_precision_score(y_test, y_score_svc)

In [ ]:
y_score_rf = best_rf.predict_proba(X_test)[:, 1]
precision_rf, recall_rf, _ = precision_recall_curve(y_test, y_score_rf)
auc_rf = auc(recall_rf, precision_rf)
avg_prec_rf = average_precision_score(y_test, y_score_rf)

In [ ]:
plt.figure(figsize=(8, 6))

plt.plot(recall_logreg, precision_logreg, 
         label=f'LogReg (AUC = {auc_logreg:.2f}, AP = {avg_prec_logreg:.2f})')
plt.plot(recall_svc, precision_svc, 
         label=f'SVC (AUC = {auc_svc:.2f}, AP = {avg_prec_svc:.2f})')
plt.plot(recall_rf, precision_rf, 
         label=f'Random Forest (AUC = {auc_rf:.2f}, AP = {avg_prec_rf:.2f})')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend(loc='lower left')
plt.grid(True)
plt.xlim([0.0, 1.05])
plt.ylim([0.0, 1.05])
plt.show()